# **Incremental Parameter**

In [1]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, 04fcd2dc-1400-413a-a2c5-55fd128aa9ad, 3, Finished, Available, Finished)

# **CREATING DIMENSION -date**

In [2]:
df_src=spark.sql('''
SELECT DISTINCT(Date_ID),Day,Month,Year
FROM parquet.`abfss://incre_ws@onelake.dfs.fabric.microsoft.com/lake_san.Lakehouse/Files/silver/carsales`
''')

StatementMeta(, 04fcd2dc-1400-413a-a2c5-55fd128aa9ad, 4, Finished, Available, Finished)

In [3]:
display(df_src)

StatementMeta(, 04fcd2dc-1400-413a-a2c5-55fd128aa9ad, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b071172e-8f8e-4eb7-86c4-c7322a885383)

## initial and incremental

In [4]:
if spark.catalog.tableExists("dim_date"):
    df_sink=spark.sql('''
    SELECT dim_date_key,Date_ID,Day,Month,Year
    FROM dim_date''')
else:
    df_sink=spark.sql('''
    SELECT 1 AS dim_date_key,Date_ID,Day,Month,Year
    FROM parquet.`abfss://incre_ws@onelake.dfs.fabric.microsoft.com/lake_san.Lakehouse/Files/silver/carsales`
    WHERE 1=0 ''') #creating empty table with schema

StatementMeta(, 04fcd2dc-1400-413a-a2c5-55fd128aa9ad, 6, Finished, Available, Finished)

In [5]:
df_filter=df_src.join(df_sink,df_src['Date_ID']==df_sink['Date_ID'],'left').select(df_src['Date_ID'],df_src['Day'],df_src['Month'],df_src['Year'],df_sink['dim_date_key'])
df_old=df_filter.filter(col('dim_date_key').isNotNull())
df_new=df_filter.filter(col('dim_date_key').isNull()).select('Date_ID', 'Day', 'Month', 'Year')

StatementMeta(, 04fcd2dc-1400-413a-a2c5-55fd128aa9ad, 7, Finished, Available, Finished)

In [6]:
display(df_new)

StatementMeta(, 04fcd2dc-1400-413a-a2c5-55fd128aa9ad, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2c30e212-8701-4df5-ba9a-00a32ade3a18)

## Surrogate key creation

In [7]:
if spark.catalog.tableExists("dim_date"):
    max_value_df=spark.sql('''SELECT MAX(dim_date_key) FROM dim_date''') #return as a dataset
    max_value=max_value_df.collect()[0][0]
else:
    max_value=1

StatementMeta(, 04fcd2dc-1400-413a-a2c5-55fd128aa9ad, 9, Finished, Available, Finished)

In [8]:
df_new = df_new.dropDuplicates(["Date_ID"])

StatementMeta(, 04fcd2dc-1400-413a-a2c5-55fd128aa9ad, 10, Finished, Available, Finished)

In [10]:
df_new = df_new.withColumn(
    "dim_date_key",
    monotonically_increasing_id() + 1)


StatementMeta(, 04fcd2dc-1400-413a-a2c5-55fd128aa9ad, 12, Finished, Available, Finished)

In [11]:
display(df_new)

StatementMeta(, 04fcd2dc-1400-413a-a2c5-55fd128aa9ad, 13, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 886a934d-9acb-4afa-92c3-5e56d11aaa9b)

In [19]:
df_final=df_new.union(df_old)
df_final=df_final.dropDuplicates(["Date_ID"])

StatementMeta(, 04fcd2dc-1400-413a-a2c5-55fd128aa9ad, 21, Finished, Available, Finished)

# **SCD TYPE-1: UPSERT**

In [20]:
from delta.tables import DeltaTable

StatementMeta(, 04fcd2dc-1400-413a-a2c5-55fd128aa9ad, 22, Finished, Available, Finished)

In [21]:
if spark.catalog.tableExists('dim_date'):            #Incremental run
    delta_table=DeltaTable.forPath(spark,'abfss://incre_ws@onelake.dfs.fabric.microsoft.com/lake_san.Lakehouse/Files/gold/dim_date')
    delta_table.alias("trg").merge(df_final.alias("src"),"trg.Date_ID=src.Date_ID")\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()
else:                                                      #Initial run
    df_final.write.format("delta")\
        .mode('overwrite')\
        .option('path','abfss://incre_ws@onelake.dfs.fabric.microsoft.com/lake_san.Lakehouse/Files/gold/dim_date')\
        .saveAsTable('dim_date')

StatementMeta(, 04fcd2dc-1400-413a-a2c5-55fd128aa9ad, 23, Finished, Available, Finished)

In [22]:
%%sql
SELECT * FROM dim_date

StatementMeta(, 04fcd2dc-1400-413a-a2c5-55fd128aa9ad, 24, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 5 fields>

In [1]:
%%sql
SELECT COUNT(*) FROM dim_date

StatementMeta(, 45da0a38-c992-4c9e-81ac-92fb363fdbe1, 2, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 1 fields>